In [2]:
import numpy as np
import pandas as pd
#import warnings
#warnings.filterwarnings('ignore')
import unicodedata
import re
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
# Carga manual de los archivos con su respectiva etiqueta
#df1 = pd.read_excel('/media/fabian/INFO/Universidad externado/ML2/proyecto/data/CAsesionatosSelectivos_202503.xlsx')
#df1['caso'] = 'Asesinatos selectivos'

#df2 = pd.read_excel('data/CDesaparicionForzada_202503.xlsx') 
#df2['caso'] = 'Desaparición forzada'

#df3 = pd.read_excel('data/CMasacres_202503.xlsx')
#df3['caso'] = 'Masacres'

df4 = pd.read_excel('/media/fabian/INFO/Universidad externado/ML2/proyecto/data/CReclyUtiliNinoNinasAdo_202503.xlsx')
df4['caso'] = 'Reclutamiento'

#df5 = pd.read_excel('data/CViolenciaSex_202503.xlsx')
#df5['caso'] = 'Violencia sexual'


/home/fabian/miniconda3/envs/islp/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
# Agrupar todos los dataframes en una lista
#dfs = [df1, df2, df3, df4, df5]

# Convertir todos los 'ID Caso' a string para comparación uniforme
#for i in range(len(dfs)):
#    dfs[i]['ID Caso Relacionado'] = dfs[i]['ID Caso Relacionado'].astype(str)

# Comparación cruzada de IDs
#for i in range(len(dfs)):
#    for j in range(i + 1, len(dfs)):
#        comunes = dfs[i]['ID Caso Relacionado'].isin(dfs[j]['ID Caso Relacionado']).sum()
#        print(f"IDs comunes entre df{i+1} y df{j+1}: {comunes}")

IDs comunes entre df1 y df2: 151732
IDs comunes entre df1 y df3: 151151
IDs comunes entre df1 y df4: 151149
IDs comunes entre df1 y df5: 151890
IDs comunes entre df2 y df3: 65425
IDs comunes entre df2 y df4: 65281
IDs comunes entre df2 y df5: 65421
IDs comunes entre df3 y df4: 3976
IDs comunes entre df3 y df5: 4042
IDs comunes entre df4 y df5: 16950


## revisamos las combinaciones y columnas en comun 

In [ ]:
#from itertools import combinations
#dfs = [df1, df2, df3, df4, df5]
#names = ['Asesinatos', 'Desaparición', 'Masacres', 'Reclutamiento', 'Violencia']

# Columnas comunes a los 5 -------------------------------------------
#common_all = set(dfs[0].columns)
#for d in dfs[1:]:
#    common_all &= set(d.columns)

#print("\n Columnas presentes en TODOS los datasets:")
#for col in sorted(common_all):
#    print("  -", col)

# Columnas comunes por pares (opcional) -------------------------------
#print("\n Columnas comunes por pareja de datasets:")
#for (i, j) in combinations(range(len(dfs)), 2):
#    col_pair = set(dfs[i].columns) & set(dfs[j].columns)
#    print(f"  {names[i]} + {names[j]}: {len(col_pair)} columnas en común")


► Columnas presentes en TODOS los datasets:
  - Abandono o Despojo Forzado de Tierras
  - Amenaza o Intimidación
  - Ataque Contra Misión Médica
  - Año
  - Confinamiento o Restricción a la Movilidad
  - Código DANE de Municipio
  - Departamento
  - Descripción Presunto Responsable
  - Desplazamiento Forzado
  - Día
  - Extorsión
  - ID Caso
  - ID Caso Relacionado
  - Latitud
  - Lesionados Civiles
  - Longitud
  - Mes
  - Modalidad
  - Municipio
  - Otro Hecho Simultáneo
  - Pillaje
  - Presunto Responsable
  - Región
  - Tortura
  - Total de Víctimas del Caso
  - Violencia Basada en Género
  - caso

► Columnas comunes por pareja de datasets:
  Asesinatos + Desaparición: 28 columnas en común
  Asesinatos + Masacres: 28 columnas en común
  Asesinatos + Reclutamiento: 27 columnas en común
  Asesinatos + Violencia: 28 columnas en común
  Desaparición + Masacres: 28 columnas en común
  Desaparición + Reclutamiento: 27 columnas en común
  Desaparición + Violencia: 28 columnas en común
  

## categorizamos las relaciones y codificamos

In [18]:
# Definir columnas de interés
cols_interes = [
    'ID Caso Relacionado', 'Año', 'Presunto Responsable',
    'Latitud', 'Longitud', 'Total de Víctimas del Caso', 'caso'
]


df4 = df4.copy()

for col in cols_interes:
    if col not in df4.columns:
        df4[col] = pd.NA

# Seleccionar únicamente las columnas en orden
df = df4[cols_interes]


In [19]:
df.shape

(17280, 7)

# Revisamos los Na y las estadisticas basicas

In [20]:
# Ver primeras filas
print("Primeras filas del conjunto de datos:")
display(df.head())

# Dimensiones del DataFrame
print(f"\nDimensiones del dataset: {df.shape[0]} filas y {df.shape[1]} columnas")

# Tipos de datos y nulos
print("\nResumen de tipos de datos y valores nulos:")
display(df.info())
display(df.isnull().sum())

# Estadísticas básicas
print("\nEstadísticas descriptivas:")
display(df.describe())

Primeras filas del conjunto de datos:


,ID Caso Relacionado,Año,Presunto Responsable,Latitud,Longitud,Total de Víctimas del Caso,caso
0,NaN,2004,GRUPO PARAMILITAR,6.219481,-74.571689,1,Reclutamiento
1,NaN,2001,GRUPO PARAMILITAR,5.837681,-74.828073,1,Reclutamiento
2,NaN,0,GUERRILLA,6.992244,-76.314263,1,Reclutamiento
3,NaN,1988,GUERRILLA,6.894292,-73.539003,1,Reclutamiento
4,NaN,1998,GUERRILLA,5.416268,-75.904326,1,Reclutamiento



Dimensiones del dataset: 17280 filas y 7 columnas

Resumen de tipos de datos y valores nulos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17280 entries, 0 to 17279
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID Caso Relacionado         375 non-null    object 
 1   Año                         17280 non-null  int64  
 2   Presunto Responsable        17280 non-null  object 
 3   Latitud                     17280 non-null  float64
 4   Longitud                    17280 non-null  float64
 5   Total de Víctimas del Caso  17280 non-null  int64  
 6   caso                        17280 non-null  object 
dtypes: float64(2), int64(2), object(3)
memory usage: 945.1+ KB


None

ID Caso Relacionado           16905
Año                               0
Presunto Responsable              0
Latitud                           0
Longitud                          0
Total de Víctimas del Caso        0
caso                              0
dtype: int64


Estadísticas descriptivas:


,Año,Latitud,Longitud,Total de Víctimas del Caso
count,17280.000000,17280.000000,17280.000000,17280.000000
mean,701.587731,4.846684,-74.372279,1.057581
std,956.135797,2.639673,1.930497,1.249798
min,0.000000,-3.530059,-81.717624,1.000000
25%,0.000000,2.975772,-75.875339,1.000000
50%,0.000000,4.005034,-74.528685,1.000000
75%,2002.000000,6.906942,-72.590385,1.000000
max,2025.000000,12.543115,-67.487095,150.000000


# Quitamos los caracteres especiales de los nombres de las columnas

In [21]:
def clean_column_name(col):
    # Elimina acentos
    col = ''.join(c for c in unicodedata.normalize('NFKD', col) if not unicodedata.combining(c))
    # Reemplaza espacios y otros caracteres especiales por guiones bajos
    col = re.sub(r'[^\w]+', '_', col)
    # Elimina guiones bajos redundantes al final o inicio
    col = col.strip('_')
    return col

df.columns = [clean_column_name(col) for col in df.columns]

df.drop(columns=['ID_Caso_Relacionado'], inplace=True)

/tmp/ipykernel_43788/4162079830.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['ID_Caso_Relacionado'], inplace=True)


In [23]:
df.drop(columns=['caso'], inplace=True)

/tmp/ipykernel_43788/3976645230.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['caso'], inplace=True)


In [24]:
df.columns

Index(['Ano', 'Presunto_Responsable', 'Latitud', 'Longitud',
       'Total_de_Victimas_del_Caso'],
      dtype='object')

## tratamiento de las categoricas

In [25]:
# Calcular el promedio de los valores distintos de 0
promedio_ano = int(round(df.loc[df['Ano'] != 0, 'Ano'].mean()))

# Reemplazar los 0 por el promedio
df.loc[df['Ano'] == 0, 'Ano'] = promedio_ano

# Confirmar resultado
print(f"Año promedio usado: {promedio_ano}")

Año promedio usado: 2005


In [26]:
vc = (
    df['Ano']
    .fillna('Ninguno')          # por si quedaban NaN ocultos
    .value_counts(dropna=False)
    .rename_axis('hecho')
    .reset_index(name='conteo')
)

# Top
print('cantidad:', len(df))
print("\nTOP-30 hechos simultáneos:")
print(vc.head(30).to_string(index=False))


cantidad: 17280

TOP-30 hechos simultáneos:
 hecho  conteo
  2005   11728
  2003     752
  2004     741
  2002     603
  2001     417
  2006     382
  2007     326
  2000     291
  2010     234
  2008     225
  2009     214
  2011     183
  2012     176
  1999     148
  2013     143
  2014     110
  1998      93
  1997      68
  2015      59
  1996      58
  1995      42
  1994      33
  1993      29
  1989      21
  2024      20
  1992      20
  1990      19
  2016      18
  2020      14
  2023      12


In [27]:
df['Presunto_Responsable'].value_counts()

Presunto_Responsable
DESCONOCIDO                              9314
GUERRILLA                                4329
GRUPO PARAMILITAR                        1856
GRUPO ARMADO NO IDENTIFICADO             1498
GRUPO POSDESMOVILIZACIÓN                  262
AGENTE DEL ESTADO                          12
OTRO ¿CUÁL?                                 4
BANDOLERISMO                                3
CRIMEN ORGANIZADO                           1
AGENTE DEL ESTADO - GRUPO PARAMILITAR       1
Name: count, dtype: int64

In [28]:
df.columns

Index(['Ano', 'Presunto_Responsable', 'Latitud', 'Longitud',
       'Total_de_Victimas_del_Caso'],
      dtype='object')

In [29]:
df.to_csv('/media/fabian/INFO/Universidad externado/ML2/proyecto/data/datos_concatenados.csv', index=False)